From https://datascience.stackexchange.com/questions/55566/tool-for-labeling-audio

In [1]:
from pathlib import Path
import pandas as pd
import librosa
import librosa.display
import numpy as np
from tqdm import tqdm


In [7]:
# build dataset pandas df
train_dataset = pd.read_pickle('../saved_features_train.pickle')
test_dataset = pd.read_pickle('../saved_features_test.pickle')

print('Finished feature extraction from ', len(train_dataset), ' files')

Finished feature extraction from  41265  files


In [8]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
x_train = np.array(train_dataset.feature.tolist())
y_train = np.array(train_dataset.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(y_train)) 

x_test = np.array(test_dataset.feature.tolist())
y_test = np.array(test_dataset.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
y_test = to_categorical(le.fit_transform(y_test)) 

In [13]:
import importlib
import simple_model

importlib.reload(simple_model)
model = simple_model.create_model()

In [14]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 256)               10496     
_________________________________________________________________
activation_10 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_11 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                

In [15]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
saved_folder = 'model_saves'
if not os.path.isdir(saved_folder):
    os.mkdir(saved_folder)

num_epochs = 300
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath=f'{saved_folder}/weights.best.basic_cnn_40.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 41265 samples, validate on 9630 samples
Epoch 1/300
41265/41265 [==============================] - 1s 17us/step - loss: 6.4064 - accuracy: 0.7885 - val_loss: 0.1969 - val_accuracy: 0.8990

Epoch 00001: val_loss improved from inf to 0.19689, saving model to model_saves/weights.best.basic_cnn_40.hdf5
Epoch 2/300
41265/41265 [==============================] - 1s 15us/step - loss: 0.5058 - accuracy: 0.8741 - val_loss: 0.1397 - val_accuracy: 0.9665

Epoch 00002: val_loss improved from 0.19689 to 0.13971, saving model to model_saves/weights.best.basic_cnn_40.hdf5
Epoch 3/300
41265/41265 [==============================] - 1s 14us/step - loss: 0.2467 - accuracy: 0.9139 - val_loss: 0.1101 - val_accuracy: 0.9673

Epoch 00003: val_loss improved from 0.13971 to 0.11010, saving model to model_saves/weights.best.basic_cnn_40.hdf5
Epoch 4/300
41265/41265 [==============================] - 1s 15us/step - loss: 0.1846 - accuracy: 0.9343 - val_loss: 0.0850 - val_accuracy: 0.9798

Epoch 00004: v

In [16]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9979885816574097
Testing Accuracy:  0.9812045693397522
